In [15]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import houseprice_functions as hpf
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn import metrics
from sklearn import impute
from sklearn.preprocessing import power_transform
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
cleaned_hp = pd.read_csv('data/cleaned_houseprice.csv')
HousePrices = pd.read_csv('data/train.csv')
cleaned_hp_test = pd.read_csv('data/cleaned_houseprice_test.csv')
y = HousePrices[["Id",'SalePrice']]

x_y = cleaned_hp.merge(y,how = "inner",on = "Id")
y_train = np.log(x_y['SalePrice'])
y_train = power_transform(x_y[['SalePrice']])
X_train = x_y.drop(["Id","SalePrice","Ext_ImStucc", "Ext_Stone","House_2.5Fin"],axis = 1)

C:\Users\Micha\Downloads\Anaconda\lib\site-packages\sklearn\preprocessing\_data.py:3116: FutureWarning: The default value of 'method' will change from 'box-cox' to 'yeo-johnson' in version 0.23. Set the 'method' argument explicitly to silence this warning in the meantime.
  FutureWarning)


In [ ]:
base_features = cleaned_hp[['TotalSF','GarageArea','KitchenQual','TotalBath','OverallScore','Ext_Score','YearRemodAdd','FireplaceQu','Basm_Quality']]
X_train = base_features

In [17]:
#outliers
train_outliers = hpf.outlier_selecter(cleaned_hp,cleaned_hp,num_sd = 4,min_unique = 20, drop_zeros = True)
test_range = hpf.outlier_selecter(cleaned_hp,cleaned_hp,num_sd = 4,min_unique = 20, drop_zeros = True,for_test = True)

# Standardization

In [11]:
columns_ = cleaned_hp.columns
for col in cleaned_hp.columns:
    if (cleaned_hp[col].nunique() > 20) and (col != "Id"):
        cleaned_hp[col] = cleaned_hp[col] + 1
        #print(cleaned_hp[col].reshape(-1,1))
        box_cox = power_transform(cleaned_hp[[col]], method='box-cox')
        cleaned_hp[[col]] = box_cox

X_train = cleaned_hp.drop(["Id","Ext_ImStucc", "Ext_Stone","House_2.5Fin"],axis = 1)

C:\Users\Micha\Downloads\Anaconda\lib\site-packages\numpy\core\_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


In [12]:
X_train.head()

,CentralAir,HeatingQC,garage_score,Heating,Electrical,GarageArea,finishedbsmt,TotalSF,2Types,Basment,...,Ext_Plywood,Ext_Stucco,Ext_WdSdng,Ext_WdShing,House_1.5Fin,House_1.5Unf,House_2.5Unf,House_2Story,House_SFoyer,House_SLvl
0,1.0,5.0,6.0,1.0,5.0,0.395813,1.055500,0.094904,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,5.0,6.0,1.0,5.0,-0.003137,0.930045,0.039098,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,5.0,6.0,1.0,5.0,0.661664,0.289614,0.277295,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,4.0,6.0,1.0,5.0,0.810361,-0.376041,-0.029373,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,5.0,6.0,1.0,5.0,1.635962,0.405769,1.046535,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
cleaned_hp.loc[:,"LotArea"] = np.log(cleaned_hp.loc[:,"LotArea"])
cleaned_hp_test.loc[:,"LotArea"] = np.log(cleaned_hp_test.loc[:,"LotArea"])

# Outlier imputation

### RANDOM

###### Train

In [ ]:
for col, idx in train_outliers.items():
    hpf.outlier_imputation(cleaned_hp, cleaned_hp,idx, col = col, method = "random", decimals = 2,drop_zeros = True)
    
y_train = np.log(x_y['SalePrice'])
X_train = x_y.drop(["Id","SalePrice","Ext_ImStucc", "Ext_Stone","House_2.5Fin"],axis = 1)

###### Test

In [ ]:
for key,value in test_range.items():
    outliers = cleaned_hp_test.index[cleaned_hp_test.loc[:,key].apply(lambda x: (x < value[0]) or (x > value[1]))].to_list()
    if len(outliers) != 0:
        hpf.outlier_imputation(cleaned_hp, cleaned_hp_test,outliers, col = key, method = "random", decimals = 2,drop_zeros = True)

---

### KNeighbors

# KNN USING SKLEARN

In [8]:
KNN = impute.KNNImputer
KNN = KNN()

##### Train

In [9]:
#replaces outliers with NAN
for col, idx in train_outliers.items():
    cleaned_hp.loc[:,col].iloc[idx] = np.nan

columns_ = cleaned_hp.columns
cleaned_hp = KNN.fit_transform(cleaned_hp)
cleaned_hp = pd.DataFrame(cleaned_hp,columns = columns_)
cleaned_hp.head()

C:\Users\Micha\Downloads\Anaconda\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Id,CentralAir,HeatingQC,garage_score,Heating,Electrical,GarageArea,finishedbsmt,TotalSF,2Types,...,Ext_Stucco,Ext_WdSdng,Ext_WdShing,House_1.5Fin,House_1.5Unf,House_2.5Fin,House_2.5Unf,House_2Story,House_SFoyer,House_SLvl
0,1.0,1.0,5.0,6.0,1.0,5.0,548.0,0.824766,2566.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2.0,1.0,5.0,6.0,1.0,5.0,460.0,0.774960,2524.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,1.0,5.0,6.0,1.0,5.0,608.0,0.528261,2706.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4.0,1.0,4.0,6.0,1.0,5.0,642.0,0.285714,2473.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5.0,1.0,5.0,6.0,1.0,5.0,836.0,0.572052,3343.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


##### Test

In [10]:
#replaces outliers with NAN
for col,idx in test_range.items():
    outliers = cleaned_hp_test.index[cleaned_hp_test.loc[:,col].apply(lambda x: (x < idx[0]) or (x > idx[1]))].to_list()
    if len(outliers) != 0:
        cleaned_hp_test.loc[:,col].iloc[outliers] = np.nan

columns_ = cleaned_hp_test.columns
cleaned_hp_test = KNN.fit_transform(cleaned_hp_test)
cleaned_hp_test = pd.DataFrame(cleaned_hp_test,columns = columns_)
cleaned_hp_test.head()

C:\Users\Micha\Downloads\Anaconda\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Id,CentralAir,HeatingQC,garage_score,Heating,Electrical,GarageArea,finishedbsmt,TotalSF,2Types,...,Ext_Plywood,Ext_Stucco,Ext_WdSdng,Ext_WdShing,House_1.5Fin,House_1.5Unf,House_2.5Unf,House_2Story,House_SFoyer,House_SLvl
0,1461.0,1.0,3.0,6.0,1.0,5.0,730.0,0.693878,1778.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1462.0,1.0,3.0,6.0,1.0,5.0,312.0,0.694507,2658.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1463.0,1.0,4.0,6.0,1.0,5.0,482.0,0.852371,2557.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1464.0,1.0,5.0,6.0,1.0,5.0,470.0,0.650108,2530.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1465.0,1.0,5.0,6.0,1.0,5.0,506.0,0.205469,2560.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#TRAIN X y
y_train = np.log(x_y['SalePrice'])
X_train = cleaned_hp.drop(["Id","Ext_ImStucc", "Ext_Stone","House_2.5Fin"],axis = 1)

---

In [ ]:
impute_dict = {
    'GarageArea':'LotArea',
    'TotalBsmtSF': 'GrLivArea',
    'GrLivArea':'TotalBsmtSF',
    'EnclosedPorch':'Total_PorchDeckSF',
    'Total_PorchDeckSF':'EnclosedPorch'
}

###### Train

In [ ]:
for col, idx in train_outliers_k.items():
    for col_impute, col_neighbor in impute_dict.items():
        if col == col_impute:
            hpf.k_neighbors(cleaned_hp,cleaned_hp,col,idx,col_neighbor,k = 5)    

#TRAIN X y
y_train = np.log(x_y['SalePrice'])
X_train = cleaned_hp.drop(["Id","Ext_ImStucc", "Ext_Stone","House_2.5Fin"],axis = 1)

###### Test

In [ ]:
for col,idx in test_range_k.items():
    outliers = cleaned_hp_test.index[cleaned_hp_test.loc[:,key].apply(lambda x: (x < value[0]) or (x > value[1]))].to_list()
    if len(outliers) != 0:
        for col_impute, col_neighbor in impute_dict.items():
            if col == col_impute:
                hpf.k_neighbors(cleaned_hp,cleaned_hp_test,col,outliers,col_neighbor,k = 5)

# Elastic Net

### standardization 

##### Train

In [14]:
en_model = ElasticNetCV(l1_ratio=[0.7], eps=1e-3, fit_intercept=True, 
                        normalize=True, precompute='auto', max_iter=2000, tol=0.0001, cv=10, 
                        copy_X=True, verbose=0, n_jobs=-1, positive=False, random_state=0)

for col in X_train.columns:
    if (X_train[col].nunique() != 2) and (X_train[col].skew() > 0.75):
        print(col)
        print(X_train[col].skew())
        X_train[col] = np.log1p(X_train[col])
        print(X_train[col].skew())
        print('-'*50)

full_model = en_model.fit(X_train, y_train)
y_pred = full_model.predict(X_train)
print("Full Model RMSE:",np.sqrt(metrics.mean_squared_error(y_pred, y_train)))
print("Full model R2:",en_model.score(X_train,y_train))

LotShape
1.309985656555955
0.7831035648282987
--------------------------------------------------
LandSlope
4.813682424489448
4.297167061742489
--------------------------------------------------
EnclosedPorch
2.0559919593921494
2.0559776598539408
--------------------------------------------------
Full Model RMSE: 0.11717843894149009
Full model R2: 0.9138879225938381


##### Test

In [ ]:
X_test = cleaned_hp_test.drop(["Id"],axis = 1)
for col in X_test.columns:
    if (X_test[col].nunique() != 2) and (X_test[col].skew() > 0.75):
        print(col)
        print(X_test[col].skew())
        X_test[col] = np.log1p(X_test[col])
        print(X_test[col].skew())
        print('-'*50)

In [ ]:
y_log_values = full_model.predict(X_test)
y_test = np.exp(y_log_values)
results = pd.DataFrame(zip(cleaned_hp_test.Id.astype(int),y_test),columns = ["Id","SalePrice"])
results.head(10)

In [ ]:
test_submission = results.to_csv('data/submission.csv',index = False)

# MLR

In [ ]:
X_test = cleaned_hp_test.drop(["Id"],axis = 1)
ols = linear_model.LinearRegression()
MLR = ols.fit(X_train,y_train)
print("Model R^2:",MLR.score(X_train,y_train))

In [ ]:
MLR_model_predictions = np.exp(MLR.predict(X_test))
MLR_model = pd.DataFrame(zip(cleaned_hp_test.Id.astype(int),MLR_model_predictions),columns = ["Id","SalePrice"])
MLR_model.head()

In [ ]:
test_submission = MLR_model.to_csv('data/submission1.csv',index = False)